In [ ]:
%matplotlib notebook

In [ ]:
# 1) Wipe out all Python variables
%reset -f
# 2) Force Python’s garbage collector to run
import gc
gc.collect()

import importlib
from libs import trades, plots, params, models
importlib.reload(trades)
importlib.reload(plots)
importlib.reload(params)
importlib.reload(models)

import math
import pandas as pd
from pandas import Timestamp
import numpy as np

import glob
import datetime as dt
from datetime import datetime

import optuna
from optuna.trial import TrialState
from optuna.importance import get_param_importances

import json

import matplotlib.pyplot as plt
from IPython.display import display, clear_output, update_display
from tqdm.auto import tqdm

pd.set_option('display.max_columns', None)

import os # Force fully single-threaded
os.environ["OMP_NUM_THREADS"]    = "1"
os.environ["MKL_NUM_THREADS"]    = "1"
os.environ["NUMEXPR_NUM_THREADS"]= "1"

In [ ]:
ticker         = params.ticker
save_path      = params.save_path

results_folder = "optuna results"              
n_trials = 300
n_jobs = 1

df_path = save_path / f"{ticker}_base.csv"
df = pd.read_csv(df_path, index_col=0, parse_dates=["datetime"])

df

In [ ]:
def optimiz_function(
    df,
    look_back,
    min_prof_thr, 
    max_down_prop, 
    gain_tightening_factor, 
    merging_retracement_thr, 
    merging_time_gap_thr, 
    smooth_win_sig, 
    pre_entry_decay, 
    short_penalty,
    trailing_stop_thresh,
    buy_threshold
):
    # 1) Compute the shifted session start for this trial
    start_min = (
        params.regular_start.hour * 60
      + params.regular_start.minute
      - 2 * look_back
    )
    regular_start_shifted = dt.time(*divmod(start_min, 60))

    print(" Step A: smoothing & DST adjust …")
    df_prep = trades.prepare_interpolate_data(
        df                    = df,
        regular_start_shifted = regular_start_shifted,
        regular_start         = params.regular_start, 
        regular_end           = params.regular_end
    )

    # 2) Compute the one‐time median profit reference *for this look_back* 
    print(" Step A1: compute global ref_profit …")
    global_ref = trades.compute_global_ref_profit(
        df                     = df_prep,
        min_prof_thr           = min_prof_thr,
        max_down_prop          = max_down_prop,
        gain_tightening_factor = gain_tightening_factor,
        merging_retracement_thr= merging_retracement_thr,
        merging_time_gap_thr   = merging_time_gap_thr,
        regular_start_shifted  = regular_start_shifted,
        regular_end            = params.regular_end
    )
    print(" global_ref =", global_ref)

    # 3) Run your pipeline once with that ref_profit
    print("\n Step B: running trading pipeline on full dataset …")
    full_sim_results = trades.run_trading_pipeline(
        df_prep                 = df_prep,
        col_signal              = 'signal_smooth',
        col_action              = 'signal_action',
        ref_profit              = global_ref,
        min_prof_thr            = min_prof_thr, 
        max_down_prop           = max_down_prop, 
        gain_tightening_factor  = gain_tightening_factor, 
        merging_retracement_thr = merging_retracement_thr, 
        merging_time_gap_thr    = merging_time_gap_thr,
        smooth_win_sig          = smooth_win_sig, 
        pre_entry_decay         = pre_entry_decay,
        short_penalty           = short_penalty,
        trailing_stop_thresh    = trailing_stop_thresh,
        buy_threshold           = buy_threshold
    )

    # 4) Aggregate returns
    strat_returns = [res[2]['Strategy Return ($)']
                     for res in full_sim_results.values()]
    sum_returns  = np.sum(strat_returns)
    mean_returns = np.mean(strat_returns)

    # 5) Clean up and return
    del df_prep, full_sim_results
    gc.collect()

    print('sum_returns:', sum_returns)
    print('mean_returns:', mean_returns)
    return mean_returns, sum_returns


In [ ]:

# === Objective Function ===
def objective(trial):
    # Suggest parameters to test.
    hyperpar = {
        "look_back" : trial.suggest_categorical("look_back", [30, 60, 90, 180]),
        "min_prof_thr": trial.suggest_float("min_prof_thr", 0.1, 1),
        "max_down_prop": trial.suggest_float("max_down_prop", 0.1, 0.9),
        "gain_tightening_factor": trial.suggest_float("gain_tightening_factor", 0.1, 1),
        "merging_retracement_thr": trial.suggest_float("merging_retracement_thr", 0.1, 1),
        "merging_time_gap_thr": trial.suggest_float("merging_time_gap_thr", 0.1, 1),
        "smooth_win_sig": trial.suggest_categorical("smooth_win_sig", [1, 2, 3, 5, 10, 15, 45, 60, 90, 120, 180, 240]),
        "pre_entry_decay": trial.suggest_float("pre_entry_decay", 0.01, 0.5),
        "short_penalty": trial.suggest_float("short_penalty", 0.05, 0.75),
        "trailing_stop_thresh": trial.suggest_float("trailing_stop_thresh", 0.05, 0.75),
        "buy_threshold": trial.suggest_float("buy_threshold", 0.05, 0.75)
    }

    print(f"\n▶ Trial {trial.number} starting with:\n{hyperpar}\n")
    
    # Run your strategy simulation with the current set of parameters.
    mean_returns, sum_returns = optimiz_function(
        df=df,
        look_back=hyperpar["look_back"],
        min_prof_thr=hyperpar["min_prof_thr"],
        max_down_prop=hyperpar["max_down_prop"],
        gain_tightening_factor=hyperpar["gain_tightening_factor"],
        merging_retracement_thr=hyperpar["merging_retracement_thr"],
        merging_time_gap_thr=hyperpar["merging_time_gap_thr"],
        smooth_win_sig=hyperpar["smooth_win_sig"],
        pre_entry_decay=hyperpar["pre_entry_decay"],
        short_penalty=hyperpar["short_penalty"],
        trailing_stop_thresh=hyperpar["trailing_stop_thresh"],
        buy_threshold=hyperpar["buy_threshold"]
    )

    gc.collect()
    
    return mean_returns



In [ ]:
# ----------------------------------------------------------
# create ONE figure
# ----------------------------------------------------------
# build blank figure & line
fig, ax = plt.subplots(figsize=(7,3))
line, = ax.plot([], [], "bo-")
ax.set(xlabel="Trial #", ylabel="Objective",
       title="Optuna optimization progress")
ax.grid(True)

# display once and grab the handle
handle = display(fig, display_id=True)
plt.close(fig)

# ask plots.py for a callback bound to these objects
live_cb = plots.make_live_plot_callback(fig, ax, line, handle)

In [ ]:
# === Create and Run the Study ===
study = optuna.create_study(
    direction="maximize"
    # pruner=MedianPruner(n_startup_trials=6, n_warmup_steps=12),
)

def cleanup_cb(study, trial):
    import gc
    gc.collect()

study.optimize(
    objective,
    n_trials=n_trials,
    n_jobs=n_jobs,       # run trials concurrently
    callbacks=[live_cb, cleanup_cb],
)

# === Print Final Results ===
print("Best Parameters:", study.best_params)
print("Best Average Improvement:", study.best_value)

# === Compute & Print Hyperparameter Importances ===
importances = get_param_importances(study)
print("\nHyperparameter importances (higher ⇒ more impact):")
for name, score in sorted(importances.items(), key=lambda x: x[1], reverse=True):
    print(f"  {name:20s} : {score:.3f}")

# ------------------------------------------------------------------
# Build a dynamic file-name .json
# ------------------------------------------------------------------
# 1) Build your session‐only DataFrame once
session_df = df.between_time(params.regular_start,
                             params.regular_end)

# 2) Derive the trading‐day boundaries
first_day = session_df.index.normalize().min()
last_day  = session_df.index.normalize().max()

# 3) Format your file name
start_date = first_day.strftime("%Y%m%d")
end_date   = last_day.strftime("%Y%m%d")
file_name  = f"{ticker}_{start_date}-{end_date}_optuna_signal_pars.json"
file_path  = os.path.join(results_folder, file_name)


# ------------------------------------------------------------------
# Dump study results (including importances)
# ------------------------------------------------------------------
with open(file_path, "w") as f:
    json.dump(
        {
            "best_params":   study.best_params,
            "best_value":    study.best_value,
            "importances":   importances,
            "trials": [
                {
                    "number": t.number,
                    "value":  t.value,
                    "params": t.params,
                    "state":  t.state.name
                }
                for t in study.trials
            ],
        },
        f,
        indent=4,
    )

print(f"\nOptuna results (and importances) saved to: {file_path}")
